In [1]:
import os
import shutil
import csv
from tqdm import tqdm
import random

In [2]:
# Params
img_path = 'D:/birds/YOLO_birds/nabirds/images'
txt_path = 'D:/birds/YOLO_birds/nabirds/labels'
bbox_path = 'D:/birds/YOLO_birds/nabirds/bounding_boxes.txt'
label_path = 'D:/birds/YOLO_birds/nabirds/image_class_labels.txt'
class_path = 'D:/birds/YOLO_birds/nabirds/classes.txt'
names_path = './data/names.txt'
train_txt = './data/birds_train.txt'
test_txt = './data/birds_test.txt'

data_dict = {}
new_labels = {}
old_labels = {}
img_names = []
label_set = set()

In [3]:
def create_labels(idx, label):
    if label not in label_set:
        label_set.add(label)
        new_labels[label] = len(new_labels.keys())
    
    old_labels[idx] = label

In [4]:
def get_bird_name(line):
    line = line[0].split()
    label = ''
    
    for i in range(1, len(line)):
        if line[i][0] == '(':
            break
        else:
            label += ' ' + line[i]
            
    label = label[1:]
    
    return line[0], label

In [5]:
cur_line = 0
with open(class_path) as class_file:
    class_reader = csv.reader(class_file, delimiter = ',')
    for line in class_reader:        
        if cur_line < 295:
            pass
        else:
            idx, label = get_bird_name(line)
            create_labels(int(idx), label)
        cur_line +=1

In [6]:
with open(label_path) as labels_file:
    labels_reader = csv.reader(labels_file, delimiter = ',')
    for line in labels_reader:
        line = line[0].split()
        img = line[0].replace('-', '')
        old_label = old_labels[int(line[1])]
        new_label = new_labels[old_label]
        data_dict[img] = [new_label]

In [7]:
with open(bbox_path) as bbox_file:
    bbox_reader = csv.reader(bbox_file, delimiter = ',')
    for line in bbox_reader:
        line = line[0].split()
        img = line[0].replace('-', '')
        x = float(line[1])
        y = float(line[2])
        w = float(line[3])
        h = float(line[4])
        x = (x + w/2)
        y = (y + h/2)
        data_dict[img].extend((x, y ,w, h))

In [8]:
len(data_dict.keys())

48562

In [9]:
count = 0
for key in data_dict.keys():
    print(key, data_dict[key])
    if count > 5:
        break
    count += 1

0000139e21dc4d0cbfe14cae3c85c829 [287, 147.0, 173.0, 128.0, 228.0]
0000d9fc4e024c06a0afa55cfb16b12b [376, 409.5, 237.0, 163.0, 298.0]
000193069d834334b255a447742edce3 [283, 283.5, 556.0, 219.0, 378.0]
0001afd499a14a67b940d419413e23b3 [51, 553.0, 291.0, 492.0, 224.0]
000332b8997c454096472f0a8495aecf [125, 526.0, 334.0, 262.0, 390.0]
000343bd521549baab9c7c97a70ac1a5 [58, 413.5, 388.5, 587.0, 357.0]
0004ff8d0cc847ee94ba43352a8b9eb4 [6, 527.5, 342.5, 221.0, 467.0]


In [10]:
# create the labels for each image
for key in tqdm(data_dict.keys()):
    f_name = txt_path + '/' + key + '.txt'
    line = ''
    for col in data_dict[key]:
        line += ' ' + str(col)
    line = line[1:]
    f = open(f_name, 'w')
    f.write(line)
    f.close()

100%|████████████████████████████████████████████████████████████████████████████| 48562/48562 [43:05<00:00, 18.78it/s]


In [10]:
# create nabirds.names
f = open(names_path, 'w')
for key in new_labels.keys():
    f.write(key + '\n')
f.close()

In [11]:
def fix_slash(path):
    path = path.split('\\')
    path = img_path + '/' + path[1]
    return path

In [12]:
num_imgs = len(data_dict.keys())
test_size = int(0.1 * num_imgs)
test_idx = random.sample(range(0, num_imgs), test_size)
for subdir, dirs, imgs in os.walk(img_path):
    for img in imgs:
        #old_path = fix_slash(subdir) +'/'+ img
        #new_path = img_path +'/'+ img
        #shutil.move(old_path, new_path)
        img_names.append(img[:-4])

f_train = open(train_txt, 'w')
f_test = open(test_txt, 'w')
random.shuffle(img_names)
for i, img in tqdm(enumerate(img_names)):
    if i in test_idx:
        f_test.write(img + '\n')
        
    else:
        f_train.write(img + '\n')
        
f_train.close()
f_test.close()

48562it [00:02, 16910.85it/s]
